In [1]:
import pandas as pd
import random
import os, sys

import numpy as np
import torch
from torch.utils.data import Subset
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

from sklearn.model_selection import StratifiedKFold
from collections import defaultdict

import torch
import torch.utils.data as data


In [2]:
#데이터 path 불러오기
train = pd.read_csv('../input/data/train/train.csv')
train_path= '../input/data/train/images/'

In [3]:
#총 이미지 데이터 18,900개에 맞게 늘리는 과정 + mask 착용여부 열에 추가
new_train = pd.DataFrame(index=range(0,18900), columns=['id', 'gender', 'race', 'age','mask', 'total', 'path'])
for index, data in train.iterrows():
    path = train.loc[index]['path']
    file_list = [f for f in os.listdir(train_path + path) if not f.startswith('.')]
    for i in range(7):
        new_train.loc[index*7 + i] = train.loc[index]
        new_train.loc[index*7 + i]['path'] = new_train.loc[index*7 + i]['path'] + '/' + file_list[i]
        if 'incorrect_mask' in file_list[i]:
            new_train.loc[index*7 + i]['mask'] = 1
        elif 'mask' in file_list[i]:
            new_train.loc[index*7 + i]['mask'] = 0
        else:
            new_train.loc[index*7 + i]['mask'] = 2

In [4]:
new_train['gender'].loc[new_train.gender == 'male'] = 0
new_train['gender'].loc[new_train.gender == 'female'] = 1
                    
#new_train['mask'].loc[new_train.mask == 'mask'] = 0
#new_train['mask'].loc[new_train.mask == 'Incorrect'] = 1
#new_train['mask'].loc[new_train.mask == 'Not Wear'] = 2
                    
new_train['age'].loc[new_train.age < 30] = 0
new_train['age'].loc[(new_train.age >= 30) & (new_train.age <60)] = 1   
new_train['age'].loc[new_train.age >= 60] = 2   

In [5]:
new_train['total'] = new_train['mask']*6+new_train['gender']*3+new_train['age']

In [6]:
new_train

,id,gender,race,age,mask,total,path
0,000001,1,Asian,1,2,16,000001_female_Asian_45/normal.jpg
1,000001,1,Asian,1,1,10,000001_female_Asian_45/incorrect_mask.jpg
2,000001,1,Asian,1,0,4,000001_female_Asian_45/mask2.jpg
3,000001,1,Asian,1,0,4,000001_female_Asian_45/mask3.jpg
4,000001,1,Asian,1,0,4,000001_female_Asian_45/mask4.jpg
...,...,...,...,...,...,...,...
18895,006959,0,Asian,0,0,0,006959_male_Asian_19/mask2.jpg
18896,006959,0,Asian,0,0,0,006959_male_Asian_19/mask3.jpg
18897,006959,0,Asian,0,0,0,006959_male_Asian_19/mask4.jpg
18898,006959,0,Asian,0,0,0,006959_male_Asian_19/mask1.jpg


In [7]:
if 'race' in new_train.columns:
    new_train.drop('race', axis=1)

In [ ]:
class ImbalancedDatasetSampler(data.sampler.Sampler):
    def __init__(self, dataset: AllAgeFacesDataset):
        self.num_samples = len(dataset)

        # distribution of classes in the dataset
        label_count = defaultdict(int)

        for label in dataset.labels:
            label_count[label['age']] += 1

        self.weights = [label_count[label['age']] for label in dataset.labels]
        self.weights = 1.0 / torch.DoubleTensor(self.weights)

    def __iter__(self):
        indices = torch.multinomial(self.weights, self.num_samples, replacement=True)
        yield from indices.tolist()

    def __len__(self):
        return self.num_samples

In [ ]:
import logging
import functools

from torch.utils import data

from .imbalanced_sampler import ImbalancedDatasetSampler


def create_data_loaders(train_dataset: data.Dataset, val_dataset: data.Dataset, num_workers: int, batch_size: int):
    logging.info(f'creating dataloaders with {num_workers} workers and a batch-size of {batch_size}')

    fn_dataloader = functools.partial(
        data.DataLoader,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
    )

    imbalanced_sampler = ImbalancedDatasetSampler(train_dataset)
    train_loader = fn_dataloader(train_dataset, sampler=imbalanced_sampler)

    train_metrics_sampler = data.RandomSampler(train_dataset, replacement=True, num_samples=len(val_dataset))
    train_metrics_loader = fn_dataloader(train_dataset, sampler=train_metrics_sampler)

    val_metrics_loader = fn_dataloader(val_dataset)

    return train_loader, train_metrics_loader, 